In [132]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
from pymongo import MongoClient
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import word_tokenize
import gensim
from pprint import pprint
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import spacy

In [30]:
client = MongoClient('localhost',27017)

In [31]:
db = client['covid']
studies = db.studies 

In [4]:
doc_path = '/Users/jeloretizo/Library/Mobile Documents/com~apple~CloudDocs/CORD-19-research-challenge'

In [5]:
metadata = pd.read_csv(doc_path + '/metadata.csv')

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [115]:
metadata.loc[metadata['sha'] == 'ff766484e50701bbb36ed45d3a4464d817a471ff']['abstract'].iloc[0]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
40896,nl37p9vp,ff766484e50701bbb36ed45d3a4464d817a471ff,Elsevier,Middle East Respiratory Syndrome Interpreted: ...,10.1016/j.amjmed.2016.04.030,PMC7124269,27215907.0,els-covid,NaN,2016-09-30,"Lau, Susanna K.P.; Chan, Jasper F.W.; Hung, Iv...",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.amjmed.2016.04.030


In [70]:
comm_pmc_json = doc_path + '/comm_use_subset/comm_use_subset/pmc_json/'
comm_pdf_json = doc_path + '/comm_use_subset/comm_use_subset/pdf_json/'
custom_pmc_json = doc_path + '/custom_license/custom_license/pmc_json/'
custom_pdf_json = doc_path + '/custom_license/custom_license/pdf_json/'

In [71]:
comm_pmc_json_files = [f for f in listdir(comm_pmc_json) if isfile(join(comm_pmc_json, f))]
comm_pdf_json_files = [f for f in listdir(comm_pdf_json) if isfile(join(comm_pdf_json, f))]
custom_pmc_json_files = [f for f in listdir(custom_pmc_json) if isfile(join(custom_pmc_json, f))]
custom_pdf_json_files = [f for f in listdir(custom_pdf_json) if isfile(join(custom_pdf_json, f))]

In [8]:
len(comm_pdf_json_files)

9524

In [72]:
len(comm_pmc_json_files)

9148

In [74]:
# comm_pmc_json_files

In [10]:
len(custom_pdf_json_files)

26505

In [11]:
len(custom_pmc_json_files)

7797

In [82]:
def load_files(files_list, files_path):
    for orig_file in tqdm(files_list):
        with open((files_path + orig_file), 'r') as file:
            try:
                file = json.load(file)
    #             print(custom_pdf_json_file)
            except:
                print('Warning! File not read:', orig_file)
            res = {}
            try:
                res['paper_id'] = file['paper_id'] 
            except:
                res['paper_id'] = None

            try:
                res['abstract'] = file['abstract'][0]['text']
            except:
                res['abstract'] = None

            try: 
                res['title'] = file['metadata']['title']
            except:
                res['title'] = None

            try:
                res['authors'] = file['metadata']['authors']
            except:
                res['authors'] = None

            try:
                res['other_metadata'] = file['metadata']
            except:
                res['other_metadata'] = None

            try:
                res['other_abstract'] = file['abstract']
            except:
                res['other_abstract'] = None

    #         data = {}
            full_text = ''
            try:
    #             for i in file['body_text']:
    #                 section = i['section'].lower()
    #                 if '.' in section:
    #                     section = section.replace('.', ' ')
    #                 if section in list(data.keys()): 
    #                     new_text = data[section] + "\n" + i['text']
    #                     data[section] = new_text
    #     #                 new_text
    #     #                 data[i['section']].append(i['text'])
    #             #         print('a')
    #                 else:
    #                     data[section] = i['text']
                for i in file['body_text']:
                    full_text += i['text']

            except:
                full_text = None

            try:
                res['body'] = full_text
            except:
                res['body'] = None
            try:
                studies.insert_one(res, bypass_document_validation=True)
            except:
                print('Warning! File not inserted:', orig_file)

In [83]:
load_files(custom_pdf_json_files, custom_pdf_json)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Warning! File not read: ffc89f2bd7f513dae147b4bd0043164db16e8dc0.json
Warning! File not read: ff966b5218e3c9433966d622a806710b820a60d6.json
Warning! File not read: ff9398e9978220ab54cd85a98920d90b05c190d6.json
Warning! File not read: ffb877d67fd30e14677580c292ca9caf64659987.json
Warning! File not read: ff6c4e420ae293c8130a3a71a1ab9e466ccf0b40.json
Warning! File not read: ff765aa3a876d263d37993712a503bc2a9f5ef0c.json
Warning! File not read: ff740fd015f63477d7b236d0ffee2b7639b6dcb0.json
Warning! File not read: ffc93dea5a5f4e6e65ad155e87aafa98aa8c54cf.json
Warning! File not read: ff80b3d9f398a7e1b51c1c2dc6fdbc97969d24f0.json
Warning! File not read: ffd3606ed51d3129db5863b2246ec587e07323c8.json
Warning! File not read: ffb77680c0f79491003331d1e3d9dde3c4fcdfd9.json
Warning! File not read: ffab348b231771ee27738a4426a87a6a04f3994f.json
Warning! File not read: ff7d49ac4008f60ef9c5a437e0d504dcefd1246f.json
Warning! File not read: ffd73c393641618860486e897197311f630bf67f.json
Warning! File not re

In [84]:
load_files(custom_pmc_json_files, custom_pmc_json)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Warning! File not read: PMC1383626.xml.json
Warning! File not read: PMC1850917.xml.json
Warning! File not read: PMC1635791.xml.json
Warning! File not read: PMC1160239.xml.json
Warning! File not read: PMC1518840.xml.json
Warning! File not read: PMC1182165.xml.json
Warning! File not read: PMC2259160.xml.json
Warning! File not read: PMC2206501.xml.json
Warning! File not read: PMC1592674.xml.json
Warning! File not read: PMC1566363.xml.json
Warning! File not read: PMC1769406.xml.json
Warning! File not read: PMC2435248.xml.json
Warning! File not read: PMC1474450.xml.json
Warning! File not read: PMC1278949.xml.json
Warning! File not read: PMC1074749.xml.json
Warning! File not read: PMC2589215.xml.json
Warning! File not read: PMC1821001.xml.json
Warning! File not read: PMC2516372.xml.json
Warning! File not read: PMC156578.xml.json
Warning! File not read: PMC1519403.xml.json
Warning! File not read: PMC1602263.xml.json
Warning! File not read: PMC2443186.xml.json
Warning! File not read: PMC182099

In [84]:
for i in file.keys():
    print(i)

paper_id
metadata
abstract
body_text
bib_entries
ref_entries
back_matter


In [85]:
file['paper_id']

'86a998617c077f4fe2ab26214995a3548fbc0fc5'

In [97]:
full_data = pd.DataFrame(list(studies.find(
    {}, {'_id': 0, 'authors': 0, 'other_metadata': 0, 'other_abstract': 0})))
full_data.drop('abstract', axis=1).to_csv('./all_research.csv', index=False)

In [130]:
full_data['lower_body'] = full_data['body'].str.lower()

In [135]:
full_data['lower_body_tokens'] = full_data['lower_body'].apply(lambda x: word_tokenize(str(x)))

KeyboardInterrupt: 

In [ ]:
full

In [121]:
def find_abstract(research_id):
    if research_id is not None:
        if 'PMC' in research_id:
            try:
                abstract = metadata.loc[metadata['pmcid'] == research_id]['abstract'].iloc[0]
            except:
                abstract = None
        else:
            try: 
                abstract = metadata.loc[metadata['sha'] == research_id]['abstract'].iloc[0]
            except:
                abstract = None
    else:
        abstract = None
    return abstract

In [122]:
full_data['new_abstract'] = full_data['paper_id'].apply(lambda x: find_abstract(x))

In [127]:
full_data.loc[full_data['abstract'].isnull()]

,paper_id,abstract,title,body,new_abstract
2,e0668c4b793d0cad26639b070819334a94648123,None,GP-confirmed complete Achilles tendon rupture ...,The incidence of complete Achilles tendon rupt...,NaN
8,e4df800c7ec3d4e38f9609049d8d934edae25ae1,None,SCIENCE CHINA Supporting Information,The influenza virus used in this study was H5N...,None
18,d933299806fcd4e9e8a61e7760267154926c05ee,None,,Effective communicable disease surveillance sy...,BACKGROUND: Timely notification of infectious ...
35,abb703e29b1ee7ddf60006b33de199828666d7b1,None,The Scientific Response to a Pandemic,"T omorrow, the world could face a pandemic. It...",NaN
51,12332e3ee5f8253ed8db253e1acb48ea3bc8f27f,None,Kirigia and Muthuri BMC Res Notes The fiscal v...,China is a member state of the WHO Western Pac...,OBJECTIVE: According to the WHO coronavirus di...
...,...,...,...,...,...
52969,PMC7103101,None,In Kürze,,NaN
52970,PMC7122399,None,"The Role of Other Bacteria, Fungi, and Viruses...","Bronchiectasis, a heterogeneous disease, is ch...","Bronchiectasis, a heterogeneous disease, is ch..."
52971,PMC7086622,None,Analysis of cell fusion induced by bovine coro...,,Polykaryon formation in bovine fetal spleen (B...
52972,PMC3181148,None,NKG2D contributes to efficient clearance of pi...,,Activated murine cytotoxic T cells express the...


In [144]:
file['abstract'][0]['text']

'The recent emergence of the Middle East respiratory syndrome (MERS)-CoV, a close relative of the Severe Acute respiratory syndrome (SARS)-CoV, both of which caused a lethal respiratory infection in humans, reinforces the need for further understanding of coronavirus pathogenesis and the host immune response. These viruses have evolved diverse strategies to evade and block host immune responses, facilitating infection and transmission. Pathogenesis following infection with these viruses is characterized by a marked delay in the induction of Type I interferon (IFN I) and, subsequently, by a poor adaptive immune response. Therapies that expedite IFN I induction as well as interventions that antagonize immunoevasive virus proteins are thus promising candidates for immune modulation.'

In [146]:
res = {}
res['paper_id'] = file['paper_id'] 
res['abstract'] = file['abstract'][0]['text']
data = {}
for i in file['body_text']:
#     print(i['text'])
    if i['section'] in list(data.keys()): 
        new_text = data[i['section']] + '\n' + i['text']
        data[i['section']] = new_text
#         data[i['section']].append(i['text'])
#         print('a')
    else:
        data[i['section']] = i['text']
res['body'] = data

In [147]:
studies.insert_one(res)

In [43]:
for i in file['body_text']:
    print(i['section'])
    print(i['text'])
    

Introduction
Gold nanoparticles (GNPs) have attracted significant interest as a novel platform in nanobiotechnology and biomedicine because of their convenient surface bioconjugation with molecular probes1 and their remarkable optical2 and immunological3 properties. Recently published examples include applications of GNPs to genomics, biosensorics, immunoassays, clinical chemistry, detection and control of microorganisms, cancer cell photothermolysis, targeted delivery of drugs or other substances, and optical imaging and monitoring of biological cells and tissues.4–6 Noteworthy is the fact that GNPs are being increasingly administered to animals and humans parenterally. In particular, they serve as carriers for the delivery of drugs, genetic materials, and antigens. “Colloidal metallic gold is not bio-inert”—such is the name Brown et al.
7 gave to their article so as to stress the importance of nanometer size in biological effects, even for such a seemingly inert material as gold.
Int

In [45]:
metadata.loc[metadata['pmcid'] == 'PMC5396510']['abstract']

7590    In the past decade, gold nanoparticles have at...
Name: abstract, dtype: object

In [6]:
metadata.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [7]:
metadata['source_x'].unique()

array(['PMC', 'Elsevier', 'CZI', 'WHO', 'biorxiv', 'medrxiv'],
      dtype=object)